In [65]:
import numpy as np
from scipy.optimize import linprog

In [66]:
def solve(c: np.array, A: np.array, b: np.array, B: np.array):
    n = len(c)
    m = len(b)
    assert A.shape == (m, n)
    i = 0
    while True:
        i += 1
        #1
        A_B = A[:, B]
        A_B_inv = np.linalg.inv(A_B)
        
        #2
        c_B = c[B]

        #3
        y = c_B @ A_B_inv

        #4
        kappa = np.zeros(n)
        kappa[B] = A_B_inv @ b

        #5
        if (kappa >= 0).all():
            return kappa

        #6
        j_k = np.argmax(kappa < 0)
        k = int(np.where(B == j_k)[0])

        #7
        nB = [i for i in range(n) if i not in B]
        miu = {}
        for i in nB:
            miu[i] = A_B_inv[k] @ A[:, i]

        #8
        if all([i >= 0 for i in miu.values()]):
            return 'Задача несовместна'
        
        #9
        sigma = {}
        for i in miu.keys():
            if miu[i] < 0: 
                sigma[i] = (c[i] - A[:, i] @ y) / miu[i]
        
        #10
        j_0, sigma_0 = list(sigma.items())[0]
        for key, value in sigma.items():
            if value < sigma[j_0]:
                j_0 = key
                sigma_0 = value
                break

        #11
        B[k] = j_0

In [68]:
tests = [
    {
        'c': np.array([-4, -3, -7, 0, 0]),
        'A': np.array([[-2, -1, -4, 1, 0],
                       [-2, -2, -2, 0, 1]]),
        'b': np.array([-1, -1.5]),
        'B': np.array([3, 4])
    },
    {
        'c': np.array([2, 2, 1, -10, 1, 4, -2, -3]),
        'A': np.array([[-2, -1, 1, -7, 0, 0, 0, 2],
                       [4, 2, 1, 0, 1, 5, -1, -5],
                       [1, 1, 0, -1, 0, 3, -1, 1]]),
        'b': np.array([-2, 4, 3]),
        'B': np.array([1, 4, 6])
    },
    {
        'c': np.array([12, -2, -6, 20, -18, -5, -7, -20]),
        'A': np.array([[-2, -1, 1, -7, 0, 0, 0, 2],
                       [-4, 2, 1, 0, 1, 5, -1, 5],
                       [1, 1, 0, 1, 4, 3, 1, 1]]),
        'b': np.array([-2, 8, -2]),
        'B': np.array([1, 3, 5])
    },
    {
        'c': np.array([10, -2, -38, 16, -9, -9, -5, -7]),
        'A': np.array([[-2, -1, 10, -7, 1, 0, 0, 2],
                       [-4, 2, 3, 0, 5, 1, -1, 0],
                       [1, 1, 0, 1, -4, 3, -1, 1]]),
        'b': np.array([-2, -5, 2]),
        'B': np.array([1, 7, 4])
    }
]

In [69]:
for i in range(len(tests)):
    test = tests[i]
    result1 = linprog(-test['c'], A_eq=test['A'], b_eq=test['b'])
    result2 = solve(test['c'], test['A'], test['b'], test['B'])
    
    if type(result2) is str:
        assert result1.message == 'The algorithm terminated successfully and determined that the problem is infeasible.'
    else:
        assert np.allclose(result1.x, result2)